Scikit-learn提供了数十种内置的机器学习算法和模型，称为估计器(estimator)。每个估计器都可以用其拟合方法对某些数据进行拟合

fit函数需要两个输入：
1. 样本矩阵(或设计矩阵)X。X的大小通常是(n_samples, n_features)，这意味着样本以行表示，特征以列表示
2. 目标值y对于回归任务来说是实数，对于分类任务来说是整数(或任何其他离散的值集)。对于无监督学习任务，不需要指定y。y通常是一个一维数组，其中第i个元素对应X的第i个样本(行)的目标

In [3]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)

X = [[1,2,3],   # 2 samples, 3 features (代表两个样本，三个特征)
    [11,12,13]]

y = [0,1]   # classes of each sample

clf.fit(X, y)

print(clf.predict(X))  # predict classes of the training data

print(clf.predict([[4, 5, 6], [14, 15, 16]]))  # predict classes of new data

print(clf.predict([[9,8,7], [1,2,3]]))
print(clf.predict([[9,9,9], [10,20,30]]))

[0 1]
[0 1]
[1 0]
[1 1]


机器学习工作流程通常由不同的部分组成。典型的流水线包括预处理步骤(转换数据或估算数据)和预测目标值的最终预测器

在scikit-learn中，预处理器和转换器遵循与估计器对象相同的API(它们实际上都继承自相同的BaseEstimator类)。
transformer对象没有predict方法，而是一个transform方法，输出一个新变换后的样本矩阵X

有时，用户需要对不同的功能应用不同的转换:ColumnTransformer就是为这些用例设计的

In [8]:
from sklearn.preprocessing import StandardScaler

X = [
    [0,15],
    [1,-10]
]

# scale data according to computed scaling values
StandardScaler().fit(X).transform(X)


array([[-1.,  1.],
       [ 1., -1.]])

transformer和估计器(预测器)可以组合成一个单一的统一对象:管道。管道提供了与常规估计器相同的API:它可以通过fit和predict进行拟合和预测。
正如我们稍后将看到的，使用管道还将防止数据泄漏，即暴露训练数据中的一些测试数据

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# create a pipeline object
pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)
# load the iris dataset and split it into train and test sets
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# fit the whole pipeline
pipe.fit(X_train, y_train)
# we can now use it like any other estimator
accuracy_score(pipe.predict(X_test), y_test)

0.9736842105263158

将模型拟合到某些数据上并不意味着它在未见过的数据上能很好地预测。这需要直接评估。前面已经介绍了辅助工具train_test_split，
它可以将数据集划分为训练集和测试集，但scikit-learn还提供了许多其他模型评估工具，尤其是交叉验证工具

我们在这里简要展示如何使用cross_validate辅助函数执行5折交叉验证过程。请注意，也可以手动迭代折叠，使用不同的数据分割策略，以及使用自定义评分函数。

In [9]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
X, y = make_regression(n_samples=1000, random_state=0)
lr = LinearRegression()
result = cross_validate(lr, X, y)  # defaults to 5-fold CV
result['test_score']  # r_squared score is high because dataset is easy

array([1., 1., 1., 1., 1.])

所有估计器都有可调参(文献中通常称为超参数)。估计器的泛化能力通常严重依赖于少数参数。
例如，RandomForestRegressor有一个n_estimators参数，用于确定森林中树的数量，还有一个max_depth参数，用于确定每棵树的最大深度。
通常，这些参数的确切值是多少并不清楚，因为它们依赖于手头的数据

Scikit-learn提供了自动找到最佳参数组合的工具(通过交叉验证)。
在下面的示例中，我们使用RandomizedSearchCV对象在随机森林的参数空间中随机搜索。
当搜索结束时，RandomizedSearchCV的行为就像RandomForestRegressor一样，它已经拟合了最佳参数集。

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import randint
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# define the parameter space that will be searched over
# 定义要搜索的参数空间
param_distributions = {'n_estimators': randint(1, 5),
                       'max_depth': randint(5, 10)}
# now create a searchCV object and fit it to the data
search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=0),
                            n_iter=5,
                            param_distributions=param_distributions,
                            random_state=0)
search.fit(X_train, y_train)
search.best_params_

# the search object now acts like a normal random forest estimator
# with max_depth=9 and n_estimators=4
search.score(X_test, y_test)

0.735363411343253

注意在实践中，你几乎总是希望在管道中搜索，而不是在单个估计器中搜索。
其中一个主要原因是，如果你在不使用管道的情况下对整个数据集进行预处理，然后执行任何类型的交叉验证，你将打破训练数据和测试数据之间独立的基本假设。
事实上，由于您使用整个数据集对数据进行了预处理，因此有关测试集的一些信息可用于训练集。这将导致高估估计器的泛化能力(您可以在这篇[Kaggle](https://www.kaggle.com/code/alexisbcook/data-leakage)文章中阅读更多信息)。
使用管道进行交叉验证和搜索将在很大程度上避免这个常见的陷阱

In [2]:
import sklearn
print(sklearn.datasets.get_data_home())

C:\Users\林东\scikit_learn_data
